<a href="https://colab.research.google.com/github/masterokh/Primat/blob/main/Primat4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1. Реализовать метод Гаусса с выбором ведущего элемента для решения СЛАУ

In [ ]:
import numpy as np

def gaussian_elimination(A, b):
    n = len(A)

    # Прямой ход
    for i in range(n):
        # Выбор ведущего элемента
        max_index = i
        for j in range(i+1, n):
            if abs(A[j][i]) > abs(A[max_index][i]):
                max_index = j
        A[[i, max_index]] = A[[max_index, i]]
        b[[i, max_index]] = b[[max_index, i]]

        # Приведение матрицы к треугольному виду
        for j in range(i+1, n):
            factor = A[j][i] / A[i][i]
            A[j] -= factor * A[i]
            b[j] -= factor * b[i]

    # Обратный ход
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i][i+1:], x[i+1:])) / A[i][i]

    return x

# Пример использования
A = np.array([[2, -1, 3],
              [1, 3, -2],
              [4, 1, -1]], dtype=float)

b = np.array([10, 5, 12], dtype=float)

x = gaussian_elimination(A, b)
print(x)


[3. 2. 2.]


###2. Реализовать алгоритм LU-разложения с использованием разреженно-строчного (разреженно-столбцового) формата хранения матрицы, а также метод решения СЛАУ с использованием LU-разложения.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def lu_decomposition_sparse(A):
    n = A.shape[0]
    L = csr_matrix((n, n), dtype=float)
    U = csr_matrix((n, n), dtype=float)

    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k].dot(U[:k, k].toarray().flatten())

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k].dot(U[:k, j].toarray().flatten())

        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k].dot(U[:k, k].toarray().flatten())) / U[k, k]

    return L, U

# Пример использования
A = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

# Создание разреженной матрицы в формате CSR
A_sparse = csr_matrix(A)

L, U = lu_decomposition_sparse(A_sparse)

print("L:")
print(L.toarray())
print("U:")
print(U.toarray())

# Решение СЛАУ с использованием LU-разложения
b = np.array([1, 0, -1])

# Решение СЛАУ
y = spsolve(L, b)
x = spsolve(U, y)

print("Solution:")
print(x)


L:
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]
U:
[[ 2.         -1.          0.        ]
 [ 0.          1.5        -1.        ]
 [ 0.          0.          1.33333333]]
Solution:
[ 0.5  0.  -0.5]


###3. Реализовать итерационный метод решения СЛАУ (метол Зейделя, Якоби или верхней релаксации на выбор).

In [ ]:
import numpy as np

def seidel(A, b, x0, tol=1e-6, max_iter=100):
    n = len(A)
    x = x0.copy()
    iterations = 0
    residual = np.linalg.norm(A @ x - b)

    while residual > tol and iterations < max_iter:
        for i in range(n):
            x[i] = (b[i] - A[i, :i] @ x[:i] - A[i, i+1:] @ x[i+1:]) / A[i, i]

        iterations += 1
        residual = np.linalg.norm(A @ x - b)

    return x, iterations, residual

# Пример использования
A = np.array([[4, 1, -1],
              [2, 7, 1],
              [1, -3, 12]])
b = np.array([3, 19, 31])
x0 = np.zeros_like(b)

x, iterations, residual = seidel(A, b, x0)

print("Solution:")
print(x)
print("Number of iterations:", iterations)
print("Residual:", residual)


Solution:
[1 2 3]
Number of iterations: 2
Residual: 0.0


###4. Провести исследование реализованных методов на системах с матрицами A^(k) число обусловленности которых регулируется за счет изменения диагонального преобладания. Внедиагональные элементы матрицы A^(k) выбираются случайным образом из множества

In [ ]:
import numpy as np
from numpy.linalg import cond

def generate_matrix(n, k):
    A = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                A[i, j] = np.random.choice([-4, -3, -2, -1])

        if i > 0:
            A[i, i] = -np.sum(A[i, :])
        else:
            A[i, i] = -np.sum(A[i, :]) + 10**(-k)

    return A

def generate_rhs(n):
    return np.arange(1, n + 1)

def exact_solution(n):
    return np.ones(n)

def lu_decomposition(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k].dot(U[:k, k])

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k].dot(U[:k, j])

        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k].dot(U[:k, k])) / U[k, k]

    return L, U

def solve_lu(L, U, b):
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(U, y)
    return x

def evaluate_methods(n, k):
    A = generate_matrix(n, k)
    F = generate_rhs(n)
    x_exact = exact_solution(n)

    print("Matrix A^(k):")
    print(A)
    print("Condition number:", cond(A))

    # Метод Гаусса с выбором ведущего элемента
    x_gauss = np.linalg.solve(A, F)
    print("Gauss solution:", x_gauss)
    print("Gauss error:", np.linalg.norm(x_gauss - x_exact))

    # Метод LU-разложения
    L, U = lu_decomposition(A)
    x_lu = solve_lu(L, U, F)
    print("LU solution:", x_lu)
    print("LU error:", np.linalg.norm(x_lu - x_exact))

    # Метод Зейделя
    x0 = np.zeros(n)
    x_seidel, iterations, residual = seidel(A, F, x0)
    print("Seidel solution:", x_seidel)
    print("Seidel error:", np.linalg.norm(x_seidel - x_exact))
    print("Number of iterations:", iterations)
    print("Residual:", residual)
    print()

# Параметры для исследования
n = 5  # Размерность матрицы
k_values = [1, 2, 3]  # Значения k для изменения обусловленности

# Исследование методов для различных значений k
for k in k_values:
    print("==== k =", k, "====")
    evaluate_methods(n, k)

==== k = 1 ====
Matrix A^(k):
[[ 9.1 -1.  -2.  -2.  -4. ]
 [-2.  12.  -2.  -4.  -4. ]
 [-4.  -2.  10.  -1.  -3. ]
 [-2.  -3.  -3.   9.  -1. ]
 [-1.  -1.  -1.  -2.   5. ]]
Condition number: 916.4747107306417
Gauss solution: [195.51776266 197.38624339 197.11035525 197.38699924 197.95767196]
Gauss error: 438.43376470626686
LU solution: [195.51776266 197.38624339 197.11035525 197.38699924 197.95767196]
LU error: 438.43376470626686
Seidel solution: [75.26279931 76.04616869 76.12798399 76.38842443 77.04276017]
Seidel error: 168.09824712731964
Number of iterations: 100
Residual: 8.376398610571082

==== k = 2 ====
Matrix A^(k):
[[ 9.01 -2.   -2.   -4.   -1.  ]
 [-2.   11.   -1.   -4.   -4.  ]
 [-1.   -3.    9.   -4.   -1.  ]
 [-4.   -3.   -2.   10.   -1.  ]
 [-4.   -4.   -2.   -4.   14.  ]]
Condition number: 7494.622573014883
Gauss solution: [1181.97940503 1183.13987414 1183.34124714 1183.1208524  1183.18878719]
Gauss error: 2642.9297947673795
LU solution: [1181.97940503 1183.13987414 1183.341

Из предоставленных данных видно, что при увеличении числа обусловленности матриц A^(k) увеличивается ошибка решения для всех трех методов: метода Гаусса, LU-разложения и метода Зейделя. Это связано с тем, что высокая обусловленность матрицы указывает на то, что даже небольшие изменения в правой части системы могут привести к значительным изменениям в решении. В результате получаем большие ошибки решения.

Кроме того, можно заметить, что метод Зейделя сходится медленнее, чем метод Гаусса и LU-разложение. Это видно по количеству итераций, которое для метода Зейделя составляет 100 на каждом шаге k. Это может быть связано с особенностями выбранного начального приближения и условиями сходимости метода Зейделя.

Также следует отметить, что остатки (residuals) для всех трех методов остаются относительно стабильными на каждом шаге k.

В целом, можно сделать вывод, что при увеличении числа обусловленности матрицы решение системы линейных уравнений становится менее точным, и ошибка решения увеличивается для всех методов.

###5. Оценить зависимость числа обусловленности и точности полученного решения в зависимости от параметра k.

In [ ]:
import numpy as np
from numpy.linalg import cond

def generate_matrix(n, k):
    A = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                A[i, j] = np.random.choice([-4, -3, -2, -1])

        if i > 0:
            A[i, i] = -np.sum(A[i, :])
        else:
            A[i, i] = -np.sum(A[i, :]) + 10**(-k)

    return A

def generate_rhs(n):
    return np.arange(1, n + 1)

def exact_solution(n):
    return np.ones(n)

def lu_decomposition(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k].dot(U[:k, k])

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k].dot(U[:k, j])

        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k].dot(U[:k, k])) / U[k, k]

    return L, U

def solve_lu(L, U, b):
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(U, y)
    return x

def evaluate_methods(n, k):
    A = generate_matrix(n, k)
    F = generate_rhs(n)
    x_exact = exact_solution(n)

    print("Matrix A^(k):")
    print(A)
    print("Condition number:", cond(A))

    # Метод Гаусса с выбором ведущего элемента
    x_gauss = np.linalg.solve(A, F)
    gauss_error = np.linalg.norm(x_gauss - x_exact)
    print("Gauss error:", gauss_error)

    # Метод LU-разложения
    L, U = lu_decomposition(A)
    x_lu = solve_lu(L, U, F)
    lu_error = np.linalg.norm(x_lu - x_exact)
    print("LU error:", lu_error)

    # Метод Зейделя
    x0 = np.zeros(n)
    x_seidel, iterations, residual = seidel(A, F, x0)
    seidel_error = np.linalg.norm(x_seidel - x_exact)
    print("Seidel error:", seidel_error)

    return cond(A), gauss_error, lu_error, seidel_error

# Параметры для оценки
n = 5  # Размерность матрицы
k_values = [1, 2, 3, 4, 5]  # Значения k для изменения обусловленности

condition_numbers = []
gauss_errors = []
lu_errors = []
seidel_errors = []

# Оценка для каждого значения k
for k in k_values:
    print("==== k =", k, "====")
    cond_number, gauss_error, lu_error, seidel_error = evaluate_methods(n, k)
    condition_numbers.append(cond_number)
    gauss_errors.append(gauss_error)
    lu_errors.append(lu_error)
    seidel_errors.append(seidel_error)
    print()

# Вывод результатов
print("Condition numbers:", condition_numbers)
print("Gauss errors:", gauss_errors)
print("LU errors:", lu_errors)
print("Seidel errors:", seidel_errors)


==== k = 1 ====
Matrix A^(k):
[[ 9.1 -2.  -2.  -2.  -3. ]
 [-3.  14.  -3.  -4.  -4. ]
 [-1.  -3.   7.  -2.  -1. ]
 [-1.  -2.  -3.   9.  -3. ]
 [-1.  -4.  -2.  -1.   8. ]]
Condition number: 1310.4338618594427
Gauss error: 540.4953641934203
LU error: 540.4953641934378
Seidel error: 137.40454634097551

==== k = 2 ====
Matrix A^(k):
[[ 8.01 -1.   -4.   -1.   -2.  ]
 [-4.   12.   -2.   -4.   -2.  ]
 [-1.   -1.    5.   -1.   -2.  ]
 [-4.   -1.   -3.   11.   -3.  ]
 [-3.   -4.   -4.   -3.   14.  ]]
Condition number: 8657.468113172787
Gauss error: 2886.539069809679
LU error: 2886.5390698094016
Seidel error: 143.68265254860236

==== k = 3 ====
Matrix A^(k):
[[10.001 -2.    -2.    -3.    -3.   ]
 [-3.    12.    -1.    -4.    -4.   ]
 [-1.    -3.    10.    -4.    -2.   ]
 [-3.    -4.    -4.    13.    -2.   ]
 [-3.    -2.    -3.    -2.    10.   ]]
Condition number: 86836.93494067308
Gauss error: 34064.49760153677
LU error: 34064.49760147183
Seidel error: 125.9725893190686

==== k = 4 ====
Matrix A

Из предоставленных данных видно, что с увеличением параметра k, который отвечает за диагональное преобладание матрицы, числа обусловленности матриц A^(k) также увеличиваются. Это означает, что матрицы становятся более плохо обусловленными, и решение системы линейных уравнений становится более чувствительным к погрешностям в данных или округлении.

Как следствие, ошибка решения системы уравнений с помощью метода Гаусса и LU-разложения также увеличивается с увеличением числа обусловленности. Это отражено в значениях ошибок, которые растут с каждым шагом k.

Однако метод Зейделя демонстрирует более стабильную ошибку в решении, независимо от значения k. Это может быть связано с итерационным характером метода Зейделя, который позволяет достигать определенной точности независимо от числа обусловленности.

Таким образом, можно сделать вывод, что чем выше числа обусловленности матрицы, тем более неустойчивыми и неточными становятся решения системы линейных уравнений при использовании прямых методов, таких как метод Гаусса и LU-разложение. В то же время, метод Зейделя остается относительно стабильным при различных значениях числа обусловленности.

###6. Провести аналогичные исследования на матрицах Гильберта, которые строятся согласно формуле

In [ ]:
import numpy as np
from numpy.linalg import cond
from scipy.linalg import hilbert

def generate_rhs(n):
    return np.arange(1, n + 1)

def exact_solution(n):
    return np.ones(n)

def lu_decomposition(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k].dot(U[:k, k])

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k].dot(U[:k, j])

        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k].dot(U[:k, k])) / U[k, k]

    return L, U

def solve_lu(L, U, b):
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(U, y)
    return x

def evaluate_methods(n):
    A = hilbert(n)
    F = generate_rhs(n)
    x_exact = exact_solution(n)

    print("Matrix Hilbert(n):")
    print(A)
    print("Condition number:", cond(A))

    # Метод Гаусса с выбором ведущего элемента
    x_gauss = np.linalg.solve(A, F)
    gauss_error = np.linalg.norm(x_gauss - x_exact)
    print("Gauss error:", gauss_error)

    # Метод LU-разложения
    L, U = lu_decomposition(A)
    x_lu = solve_lu(L, U, F)
    lu_error = np.linalg.norm(x_lu - x_exact)
    print("LU error:", lu_error)

    # Метод Зейделя
    x0 = np.zeros(n)
    x_seidel, iterations, residual = seidel(A, F, x0)
    seidel_error = np.linalg.norm(x_seidel - x_exact)
    print("Seidel error:", seidel_error)

    return cond(A), gauss_error, lu_error, seidel_error

# Параметры для оценки
n_values = [3, 4, 5, 6]  # Размерности матрицы Гильберта

condition_numbers = []
gauss_errors = []
lu_errors = []
seidel_errors = []

# Оценка для каждой размерности n
for n in n_values:
    print("==== n =", n, "====")
    cond_number, gauss_error, lu_error, seidel_error = evaluate_methods(n)
    condition_numbers.append(cond_number)
    gauss_errors.append(gauss_error)
    lu_errors.append(lu_error)
    seidel_errors.append(seidel_error)
    print()

# Вывод результатов
print("Condition numbers:", condition_numbers)
print("Gauss errors:", gauss_errors)
print("LU errors:", lu_errors)
print("Seidel errors:", seidel_errors)


==== n = 3 ====
Matrix Hilbert(n):
[[1.         0.5        0.33333333]
 [0.5        0.33333333 0.25      ]
 [0.33333333 0.25       0.2       ]]
Condition number: 524.0567775860644
Gauss error: 285.6676390492988
LU error: 285.6676390492988
Seidel error: 242.93521497633745

==== n = 4 ====
Matrix Hilbert(n):
[[1.         0.5        0.33333333 0.25      ]
 [0.5        0.33333333 0.25       0.2       ]
 [0.33333333 0.25       0.2        0.16666667]
 [0.25       0.2        0.16666667 0.14285714]]
Condition number: 15513.73873892924
Gauss error: 3236.7619622087523
LU error: 3236.7619622077077
Seidel error: 347.27992886821045

==== n = 5 ====
Matrix Hilbert(n):
[[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
Condition number: 476607.2502425855
Gauss erro

Из представленных данных видно, что числа обусловленности матриц Гильберта быстро растут с увеличением размерности матрицы n. Это указывает на то, что матрицы Гильберта являются плохо обусловленными и приводят к большим ошибкам при решении систем линейных уравнений.

Метод Гаусса и LU-разложение показывают сходные значения ошибок, которые также возрастают с увеличением размерности матрицы. Это говорит о том, что оба метода чувствительны к высокому числу обусловленности и дают сопоставимую точность решения системы уравнений.

Метод Зейделя также демонстрирует увеличение ошибки с увеличением размерности матрицы, но ошибка остается сопоставимой с ошибками методов Гаусса и LU-разложения. Это может быть связано с тем, что метод Зейделя имеет итерационный характер и может достичь определенной точности, несмотря на высокое число обусловленности матрицы.

Таким образом, матрицы Гильберта являются плохо обусловленными, что ведет к большим ошибкам при решении систем линейных уравнений. Метод Зейделя остается относительно стабильным при различных размерностях матрицы, но также не избавлен от ошибок.

###7. Сравнить между собой прямые и итерационные методы по эффективности методов в зависимости от размеров n матрицы:

In [ ]:
import numpy as np
from scipy.linalg import hilbert

def generate_rhs(n):
    return np.arange(1, n + 1)

def exact_solution(n):
    return np.ones(n)

def solve_gauss(A, b):
    n = A.shape[0]
    Ab = np.hstack((A, b[:, np.newaxis]))

    for i in range(n):
        max_row = np.argmax(np.abs(Ab[i:, i])) + i

        # Обменять строки
        Ab[[i, max_row]] = Ab[[max_row, i]]

        # Элиминация
        for j in range(i+1, n):
            factor = Ab[j, i] / Ab[i, i]
            Ab[j] -= factor * Ab[i]

    x = np.zeros(n)

    # Обратная подстановка
    for i in range(n-1, -1, -1):
        x[i] = (Ab[i, -1] - np.dot(Ab[i, :-1], x)) / Ab[i, i]

    return x

def seidel(A, b, x0, max_iterations=1000, tolerance=1e-6):
    n = A.shape[0]
    x = x0.copy()
    residual = np.inf
    iterations = 0

    while residual > tolerance and iterations < max_iterations:
        x_new = np.zeros_like(x)

        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

        residual = np.linalg.norm(x_new - x)
        x = x_new
        iterations += 1

    return x, iterations, residual

# Параметры для сравнения
n_values = [10, 50, 100, 1000]

# Замер времени выполнения прямого метода (Гаусса)
gauss_times = []

for n in n_values:
    A = hilbert(n)
    F = generate_rhs(n)

    start_time = time.time()
    x_gauss = solve_gauss(A, F)
    gauss_time = time.time() - start_time
    gauss_times.append(gauss_time)

# Замер времени выполнения итерационного метода (Зейделя)
seidel_times = []

for n in n_values:
    A = hilbert(n)
    F = generate_rhs(n)
    x0 = np.zeros(n)

    start_time = time.time()
    x_seidel, _, _ = seidel(A, F, x0)
    seidel_time = time.time() - start_time
    seidel_times.append(seidel_time)

# Вывод результатов
print("=== Прямой метод (Гаусс) ===")
print("Гаусс times:", gauss_times)
print("=== Итерационный метод (Зейдель) ===")
print("Seidel times:", seidel_times)


=== Прямой метод (Гаусс) ===
Гаусс times: [0.0007762908935546875, 0.010235071182250977, 0.03869962692260742, 2.627546787261963]
=== Итерационный метод (Зейдель) ===
Seidel times: [0.08020997047424316, 0.34917330741882324, 0.6932961940765381, 8.800216913223267]


Из полученных результатов видно, что время выполнения прямого метода (Гаусса) растет значительно медленее, чем время выполнения итерационного метода (Зейделя), с увеличением размерности матрицы.